https://jump.dev/JuMP.jl/stable/manual/callbacks/

↑URLのJuliaのコードを改変したもの

In [ ]:
import cplex
from cplex.callbacks import LazyConstraintCallback


class MyLazyCallback(LazyConstraintCallback):
    def __call__(self):
        x_val = self.get_values(0)  # 変数 x の値を取得

        # `x_val` が整数でない場合はスキップ
        if abs(x_val - round(x_val)) > 1e-6:
            return

        if x_val > 2 + 1e-6:
            # x <= 2 の制約を追加
            self.add(
                constraint=cplex.SparsePair(ind=[0], val=[1.0]), sense="L", rhs=2.0
            )


# CPLEX モデルを作成
model = cplex.Cplex()
model.set_problem_type(cplex.Cplex.problem_type.MILP)

# 変数 x を追加（整数変数、最大値10）
model.variables.add(names=["x"], types=[model.variables.type.integer], ub=[10])

# 目的関数: Max x
model.objective.set_sense(model.objective.sense.maximize)
model.objective.set_linear("x", 1.0)

# Lazy Constraint コールバックを登録
model.register_callback(MyLazyCallback)

# MIP solver の設定
model.parameters.mip.limits.nodes.set(
    1
)  # Lazy constraints の適用確認のためノード探索制限

# 解く
model.solve()

# 解の出力
solution = model.solution
print("Optimal x:", solution.get_values("x"))
print("Objective value:", solution.get_objective_value())

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Limits_Nodes                        1
Legacy callback                                  LD


Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
         Disabling repeat represolve because of lazy constraint/incumbent callback.
Tried aggregator 1 time.
Reduced MIP has 0 rows, 1 columns, and 0 nonzeros.
Reduced MIP has 0 binaries, 1 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.00 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable B NodeID Parent  Depth

*     0     0      integral     0        2.0000       10.0000        0  400.00%                        0             0
Elapsed time = 0.02 sec. (0.00 ti